In [1]:
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig 
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-04-09 13:45:30.188769: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 13:45:31.118634: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 13:45:32.480806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 导入训练数据
df = pd.read_json('/mnt/workspace/dataset/train.json')
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

train_ds[:3]
val_ds[:3]

{'instruction': ['请你将以下老年人口语转换成书面语。',
  '请你将以下老年人口语转换成书面语。',
  '请你将以下老年人口语转换成书面语。'],
 'input': ['给我打电话，我说我尿不湿刚给买完就在旁边椅子上搁着',
  '有时候挣的比工资比较高一些，他也经常给老太太买点什么的，老太太爱喝点什么酒什么的，点心什么的，他也经常去买。',
  '她也有儿有女有孙子有孙女。她就在西南楼南楼。'],
 'output': ['他给我打电话时，我告诉他我刚买好尿不湿，就放在旁边的椅子上了。',
  '老三收入较高，经常给母亲买些她爱喝的酒和点心。',
  '她育有子女和孙辈，住在西南楼南楼一带。'],
 '__index_level_0__': [361, 73, 374]}

In [3]:
tokenizer = AutoTokenizer.from_pretrained('/mnt/workspace/tmp/Qwen/Qwen2___5-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='/mnt/workspace/tmp/Qwen/Qwen2___5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, si

In [4]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一位老年服务机构的文书编辑，擅长将老人的口头叙述准确、清晰地转化为正式文档。<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [5]:
tokenized_train = train_ds.map(process_func, remove_columns=train_ds.column_names)
tokenized_val = val_ds.map(process_func, remove_columns=val_ds.column_names)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [6]:
tokenizer.decode(tokenized_train[0]['input_ids'])

'<|im_start|>system\n你是一位老年服务机构的文书编辑，擅长将老人的口头叙述准确、清晰地转化为正式文档。<|im_end|>\n<|im_start|>user\n请你将以下老年人口语转换成书面语。当然老三也不错，老三他在公安系统他有这个条件。<|im_end|>\n<|im_start|>assistant\n当然老三也不错，他在公安系统工作，具备这样的条件。<|endoftext|>'

In [7]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_train[0]["labels"])))

'当然老三也不错，他在公安系统工作，具备这样的条件。<|endoftext|>'

In [8]:
import torch

model = AutoModelForCausalLM.from_pretrained(
    '/mnt/workspace/tmp/Qwen/Qwen2___5-7B-Instruct/', 
    device_map="auto",
    torch_dtype=torch.bfloat16,
    use_cache=False
)

model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [9]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [10]:
model.dtype

torch.bfloat16

# lora

In [12]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=32, # Lora 秩
    lora_alpha=64, # Lora alaph
    lora_dropout=0.1 # Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=32, target_modules={'v_proj', 'q_proj', 'up_proj', 'k_proj', 'o_proj', 'down_proj', 'gate_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [13]:
model = get_peft_model(model, config)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/mnt/workspace/tmp/Qwen/Qwen2___5-7B-Instruct/', revision=None, inference_mode=False, r=32, target_modules={'v_proj', 'q_proj', 'up_proj', 'k_proj', 'o_proj', 'down_proj', 'gate_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [14]:
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


# 配置训练参数

In [15]:
args = Seq2SeqTrainingArguments(
    output_dir="/mnt/workspace/output/Qwen2.5-7B-Instruct",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    save_on_each_node=True,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end=True,
    gradient_checkpointing=True
)

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.19.91, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2025-04-09 13:47:00,454] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: 没有那个文件或目录


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.545100,1.344462
2,1.111700,1.309181


TrainOutput(global_step=84, training_loss=1.3798025023369562, metrics={'train_runtime': 241.9038, 'train_samples_per_second': 5.581, 'train_steps_per_second': 0.347, 'total_flos': 6460681188648960.0, 'train_loss': 1.3798025023369562, 'epoch': 2.920353982300885})

In [18]:
# 导入测试数据
df = pd.read_json('/mnt/workspace/dataset/test.json')

test_ds = Dataset.from_pandas(df)

In [19]:
tokenized_test = test_ds.map(process_func, remove_columns=test_ds.column_names)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [20]:
trainer.evaluate(tokenized_test)

{'eval_loss': 1.3284080028533936,
 'eval_runtime': 4.595,
 'eval_samples_per_second': 21.545,
 'eval_steps_per_second': 2.829,
 'epoch': 2.920353982300885}

# 合并加载模型

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/mnt/workspace/tmp/Qwen/Qwen2___5-7B-Instruct/'
lora_path = '/mnt/workspace/output/Qwen2.5-7B-Instruct/checkpoint-84/' # 这里改称 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)


/usr/local/lib/python3.11/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [22]:
print(len(test_ds))  # 检查测试集样本数量
print(test_ds[0])   # 查看第一条数据内容

99
{'instruction': '请你将以下老年人口语转换成书面语。', 'input': '过去都兴外协件给加工拿来活，一般人都不接不干，干不了。', 'output': '过去工厂常将外协加工任务外包，但一般人都不敢接，因为难以胜任。'}


In [23]:
### 生成测试结果

from eval import evaluate_generation
import random
# 随机选择50条测试数据（不重复采样）
random.seed(42)  # 固定随机种子保证可复现 42
test_samples = random.sample(list(test_ds), min(50, len(test_ds)))

# 生成预测结果
predictions = []
references = []

for i, example in enumerate(test_samples):
    print(f"Processing sample {i+1}/{len(test_samples)}")  # 打印进度
    # 生成回答
    inputs = tokenizer(
        f"<|im_start|>system\n你是一位老年服务机构的文书编辑，擅长将老人的口头叙述准确、清晰地转化为正式文档。<|im_end|>\n"
        f"<|im_start|>user\n{example['instruction']}\n{example['input']}<|im_end|>\n"
        f"<|im_start|>assistant\n",
        return_tensors="pt"
    ).to(model.device)
    outputs = model.generate(**inputs, 
                             max_new_tokens=200, 
                             num_beams=1, # 禁用束搜索（加速）
                             do_sample=True, # 采样
                             temperature=0.3,      # 温度 0.1~1.5
                             top_p=0.95,            # top-p 0.7~0.95
                            )
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    predictions.append(pred)
    references.append([example['output']])  # 注意references需要是列表的列表


Processing sample 1/50
Processing sample 2/50
Processing sample 3/50
Processing sample 4/50
Processing sample 5/50
Processing sample 6/50
Processing sample 7/50
Processing sample 8/50
Processing sample 9/50
Processing sample 10/50
Processing sample 11/50
Processing sample 12/50
Processing sample 13/50
Processing sample 14/50
Processing sample 15/50
Processing sample 16/50
Processing sample 17/50
Processing sample 18/50
Processing sample 19/50
Processing sample 20/50
Processing sample 21/50
Processing sample 22/50
Processing sample 23/50
Processing sample 24/50
Processing sample 25/50
Processing sample 26/50
Processing sample 27/50
Processing sample 28/50
Processing sample 29/50
Processing sample 30/50
Processing sample 31/50
Processing sample 32/50
Processing sample 33/50
Processing sample 34/50
Processing sample 35/50
Processing sample 36/50
Processing sample 37/50
Processing sample 38/50
Processing sample 39/50
Processing sample 40/50
Processing sample 41/50
Processing sample 42/50
P

In [24]:
predictions[0]

'我挑选了一款很小号且加长的护垫，并拍摄了照片提交给他。'

In [25]:
references[0][0]

'我告诉他这是很小的护垫，但算是加长款，并拍照发给了他。'

In [26]:
import importlib
import eval
importlib.reload(eval)  # 强制重新加载
from eval import *  # 再尝试导入eval函数

import nltk
# 设置 NLTK 数据存储路径（可以是任意目录）
nltk_data_dir = "/mnt/workspace/nltk_data"

# 确保目录存在
import os
os.makedirs(nltk_data_dir, exist_ok=True)

# 将该路径添加到 NLTK 的数据搜索路径
nltk.data.path.append(nltk_data_dir)

In [27]:
# 评估
results = evaluate_generation(
    tokenizer=tokenizer,
    predictions=predictions,
    references=references,
    intensive=True,  # 中文需要设为True
    print_table=True
)

print(results)

  0%|          | 0/50 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.628 seconds.
Prefix dict has been built successfully.


我 挑选 了 一款 很小 号且 加长 的 护垫 ， 并 拍摄 了 照片 提交 给 他 。
[['我 告诉 他 这 是 很小 的 护垫 ， 但 算是 加长 款 ， 并 拍照 发给 了 他 。']]
------------------


100%|██████████| 50/50 [00:03<00:00, 16.16it/s]

每次 见到 那位 律师 时 ， 我 总 忍不住 向 她 倾诉 心中 的 苦楚 ， 因为 实在 无法 向 其他人 诉说 。
[['现在 每次 见到 律师 ， 我 都 想 倾诉 苦衷 ， 但 这些 话 实在 无法 向 他人 诉说 。']]
------------------
他 平时 喜欢 做些 需要 动手 动脑 的 事情 ， 但 性格 上 却 不 太 让 人 省心 。
[['其他 事情 他 都 喜欢 亲力亲为 追求 细致 ， 但 做事 不够 稳妥 。']]
------------------
起初 我 没有 注意 到 ， 打开 后 就 随手 把 纸盒 扔进 了 小区 的 垃圾桶 。
[['我 解释 说 一 开始 没 留意 ， 拆开 后 就 把 纸盒 扔进 垃圾桶 了 ， 后来 丢到 了 小区 的 公共 垃圾桶 里 。']]
------------------
2008 年 后 ， 他 的 行动 变得 非常 不便 ， 我们 一直 在 家里 照顾 他 。
[['2008 年 后 ， 母亲 行动 越来越 不便 ， 一直 由 我们 在家 照料 。']]
------------------
当时 确实 耽误 了 ， 每次 看到 她 痛苦 的 样子 ， 我 都 实在 不忍心 再 让 她 受折磨 。
[['在 那 期间 ， 治疗 被 耽误 了 。 看到 他 痛苦不堪 的 样子 ， 我 也 不忍心 再 继续 折腾 他 了 。']]
------------------
说 实在话 ， 他常去 有名 的 老字号 餐馆 吃饭 ， 还 和 两位 老朋友 一起 经常 去 北京 ， 品尝 正宗 的 北京烤鸭 。
[['说实话 ， 他 经常 去 有名 的 餐馆 吃饭 。 那 时候 还有 两位 老人 和 他 一起 ， 经常 专程去 北京 吃 烤鸭 。']]
------------------
爷爷 摔伤 后 ， 2009 年 如果 我 去 外地 打工 ， 还 能 挣 不少 钱 。 当时 单位 有 岗位 空缺 ， 正 需要 我 的 专业技能 ， 但 因为 爷爷 摔断了 股骨头 ， 我 留在 家里 照顾 他 ， 所以 没有 外出 工作 。
[['2009 年 爷爷 股骨 骨折 后 ， 原本 有 同事 邀请 我 外出 工作 ， 我 的 专业技能 正是 他们 需要 的 岗位 ， 收入 也 很 可

In [ ]:
###验证模型生成结果
prompt = "...."
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一位老年服务机构的文书编辑，负责将老人的口头叙述转化为正式文档。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [41]:
from eval import evaluate_all_metrics

# 测试示例
reference_text = ["他只要想去蓟县旅游，就会去玩一趟，前前后后总共去了七次。"]
generated_text = "你想去蓟县旅游就去几次，玩了七趟你想 。"
#reference_text = references[18]
#generated_text = predictions[18]

scores = evaluate_all_metrics(tokenizer, reference_text, generated_text, intensive=True) # 如果是中文请将intensive设置为True
print(scores)

你 想 去 蓟县 旅游 就 去 几次 ， 玩 了 七趟 你 想 。
[['他 只要 想 去 蓟县 旅游 ， 就 会 去 玩 一趟 ， 前前后后 总共 去 了 七次 。']]
------------------
{'BLEU-1': 0.5926500877342693, 'BLEU-2': 0.5475035704705833, 'BLEU-3': 0.4625162625612449, 'BLEU-4': 0.3750708652558153, 'sacreBLEU': 0.2044036974896712, 'ROUGE-1': 0.6428571379591838, 'ROUGE-2': 0.19354838222684714, 'ROUGE-L': 0.5294117597750866, 'METEOR': 0.4427287581699346}
